In [1]:
import torch
import torch.nn as nn

from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import ConcatDataset, DataLoader, random_split, Sampler

In [2]:
from models_by_hand import GCNConvByHand
from helpers import CVFConfigForGCNWSuccWEIDataset

In [3]:
# class SimpleRNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(SimpleRNN, self).__init__()
#         self.hidden_size = hidden_size
        
#         # Weight matrices
#         self.W_x = nn.Linear(input_size, hidden_size, bias=False)  # Input to hidden
#         self.W_h = nn.Linear(hidden_size, hidden_size, bias=False) # Hidden to hidden
#         self.b_h = nn.Parameter(torch.zeros(hidden_size))  # Bias for hidden state
        
#         # Output layer
#         self.W_y = nn.Linear(hidden_size, output_size)  # Hidden to output

#     def forward(self, x, h_t=None):
#         """
#         x: Input sequence of shape (seq_len, input_size)
#         h_t: Initial hidden state (hidden_size,)
#         """
#         seq_len, _ = x.shape
        
#         if h_t is None:
#             h_t = torch.zeros(self.hidden_size)  # Initialize hidden state
        
#         outputs = []
        
#         for t in range(seq_len):
#             x_t = x[t]  # Get the input at time t
#             h_t = torch.tanh(self.W_x(x_t) + self.W_h(h_t) + self.b_h)  # Hidden state update
#             y_t = self.W_y(h_t)  # Compute output at time t
#             outputs.append(y_t)

#         outputs = torch.stack(outputs)  # Convert list to tensor
#         return outputs, h_t  # Return all outputs and final hidden state


# # Example Usage
# input_size = 3   # Size of input vector at each time step
# hidden_size = 5  # Number of hidden units
# output_size = 2  # Output vector size

# rnn = SimpleRNN(input_size, hidden_size, output_size)

# # Create a dummy input sequence (seq_len=4, input_size=3)
# x = torch.randn(4, input_size)  # Sequence of length 4, each with input size 3

# # Forward pass
# outputs, final_hidden_state = rnn(x)

# print("Outputs:\n", outputs)
# print("Final Hidden State:\n", final_hidden_state)


In [4]:
# rnn = nn.RNN(10, 20, 2)
# input = torch.randn(5, 3, 10)
# h0 = torch.randn(2, 3, 20)
# output, hn = rnn(input)

# output.shape

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
dataset_s_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_n7_config_rank_dataset.csv",
    "star_graph_n7_edge_index.json",
)

dataset_s_n13 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "star_graph_n13_config_rank_dataset.csv",
    "star_graph_n13_edge_index.json",
)

dataset_rr_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_random_regular_graph_n7_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n7_d4_edge_index.json",
)

dataset_plc_n7 = CVFConfigForGCNWSuccWEIDataset(
    device,
    "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n7_edge_index.json",
)

batch_size = 64

dataset_coll = [
    dataset_plc_n7,
    dataset_rr_n7,
    dataset_s_n7,
]

In [7]:
train_sizes = [int(0.95 * len(ds)) for ds in dataset_coll]
test_sizes = [len(ds) - trs for ds, trs in zip(dataset_coll, train_sizes)]

train_test_datasets = [
    random_split(ds, [tr_s, ts])
    for ds, tr_s, ts in zip(dataset_coll, train_sizes, test_sizes)
]

train_datasets = [ds[0] for ds in train_test_datasets]
test_datasets = [ds[1] for ds in train_test_datasets]

In [8]:
datasets = ConcatDataset(train_datasets)
print(f"Train Dataset size: {len(datasets):,}")

Train Dataset size: 100,293


In [9]:
class CustomBatchSampler(Sampler):
    def __init__(self, datasets: ConcatDataset, batch_size: int):
        self.datasets = datasets
        self.batch_size = batch_size

    def __iter__(self):
        last_accessed = [0] + self.datasets.cumulative_sizes[:]
        end_loop = [False for _ in range(len(self.datasets.datasets))]

        while not all(end_loop):
            for turn in range(len(self.datasets.datasets)):
                if end_loop[turn]:
                    continue

                batch_size = self.batch_size
                if (
                    last_accessed[turn] + batch_size
                    >= self.datasets.cumulative_sizes[turn]
                ):
                    batch_size = (
                        self.datasets.cumulative_sizes[turn] - last_accessed[turn]
                    )
                    end_loop[turn] = True

                yield list(range(last_accessed[turn], last_accessed[turn] + batch_size))

                last_accessed[turn] += batch_size

In [10]:
batch_sampler = CustomBatchSampler(datasets, batch_size=batch_size)
dataloader = DataLoader(datasets, batch_sampler=batch_sampler)

In [11]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, 2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_0=None):
        output, h_n = self.rnn(x, h_0)
        output = self.fc(output)
        output = torch.relu(output)
        output = global_mean_pool(output, torch.zeros(output.size(1)).to(device).long())
        return output, h_n

    def fit(self, epochs):
        criterion = torch.nn.MSELoss()
        # criterion = torch.nn.L1Loss(reduction='sum')
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.0001)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in dataloader:
                x = batch[0]
                y = batch[1]
                y = y.unsqueeze(-1)
                out, h_n = self(x[0])
                # print(out.shape, y.shape)
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            print(
                "Training set | Epoch",
                epoch,
                "| MSE Loss:",
                round((total_loss / count).item(), 4),
            )

In [ ]:
model = SimpleRNN(3, 32, 1).to(device)
print(model)
print()
print("Total parameters:", f"{sum(p.numel() for p in model.parameters()):,}")
print()
model.fit(epochs=20)

SimpleRNN(
  (rnn): RNN(3, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

Total parameters: 3,329



# Testing

In [ ]:
import csv
import datetime

f = open(f"test_results/test_result_w_succ_diff_nodes_rnn_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")}.csv", "w", newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["Dataset", "Actual", "Predicted"])

criterion = torch.nn.MSELoss()

model.eval()

with torch.no_grad():
    test_concat_datasets = ConcatDataset(test_datasets)
    test_batch_sampler = CustomBatchSampler(test_concat_datasets, batch_size=10240)
    test_dataloader = DataLoader(test_concat_datasets, batch_sampler=test_batch_sampler)

    total_loss = 0
    total_matched = 0
    count = 0
    for batch in test_dataloader:
        x = batch[0]
        y = batch[1]
        y = y.unsqueeze(-1)
        out, _ = model(x[0])
        csv_writer.writerows(
            (i, j.item(), k.item())
            for (i, j, k) in zip(
                x[2], y.detach().cpu().numpy(), out.detach().cpu().numpy()
            )
        )
        loss = criterion(out, y)
        total_loss += loss
        out = torch.round(out)
        matched = (out == y).sum().item()
        total_matched += matched
        count += 1

    print(
        "Test set",
        "| MSE loss:",
        round((total_loss / count).item(), 4),
        "| Total matched",
        total_matched,
        "out of",
        len(test_concat_datasets),
        f"(Accuracy: {round(total_matched/len(test_concat_datasets) * 100, 2)}%)",
    )

f.close()